In [1]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import time
import multiprocessing
import os
from multiprocessing import Pool
import multiprocessing.pool
import bs4 as bs
import urllib.request
import numpy as np

In [2]:
import fitz
import glob
import pandas as pd
import re
from operator import itemgetter
from itertools import groupby
from word2number import w2n
import usaddress

In [3]:
broker_pdf_list = glob.glob('C:/Users/i26184/Desktop/Broker/Broker_PDF/*.pdf')
#broker_pdf_list = glob.glob('/home/ubuntu/Broker/*.pdf')

In [4]:
max([int(re.findall(r'\d+',x)[0]) for x in broker_pdf_list])

26184

In [5]:
broker_pdf_list

['C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_1390910.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_2132865.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_2400179.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_2498722.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_2809371.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_31169.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_3233747.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_704717.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816404.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816436.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816439.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816452.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816461.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816467.pdf',
 'C:/Users/i26184/Desktop/Bro

In [6]:
len(broker_pdf_list)

113

In [7]:
def get_broker_name_CRD(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    broker_name_candiate = page1_textblock[0].split('\n')
    
    if len(broker_name_candiate)==1:
        broker_name = page1_textblock[1]
    else:
        broker_name = broker_name_candiate[1]
        
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
        
    return pd.DataFrame({'CRD Number':[CRD_number],
                         'Broker Name':[broker_name]})

In [8]:
def get_broker_name_CRD_Pool(file):
    try:
        fitz_doc = fitz.open(file)
        page1 = fitz_doc.loadPage(0)
        page1_textblock = page1.getTextBlocks()
        page1_textblock = [x[4] for x in page1_textblock]
        broker_name_candiate = page1_textblock[0].split('\n')

        if len(broker_name_candiate)==1:
            broker_name = page1_textblock[1]
        else:
            broker_name = broker_name_candiate[1]

        section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
        section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]

        if len(section_end_index) == 0:
            CRD_number = 'NO CRD Number'
        else:
            CRD_number = page1_textblock[section_end_index[0]].split()[-1]

        return (file,broker_name,CRD_number)
    except:
        print(file)
        
        return (file,'Error','Error')

1. /home/ubuntu/Broker/individual_355047.pdf - Failed to load pdf
2. https://files.brokercheck.finra.org/individual/individual_1390910.pdf, no section after report summary

start_time = time.time()
p = Pool(8)
result = p.map(get_broker_name_CRD_Pool,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

In [13]:
def get_section_info(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
    
    if len(section_start_index)==0:
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Section Titles':['No Section'],
                             'Section Pages':['No Section']})
    section_titles = page1_textblock[(section_start_index[0]+1):section_end_index[0]]
    
    section_pages = page1_textblock[(section_end_index[0]+1):(section_end_index[0]+len(section_titles)+1)]
    
    return pd.DataFrame({'CRD Number':[CRD_number]*len(section_titles),
                         'Section Titles':section_titles,
                         'Section Pages':section_pages})

In [10]:
def get_section_info_Pool(file):
    CRD_number = re.findall(r'\d+',file)[0]
    try:
        fitz_doc = fitz.open(file)
        page1 = fitz_doc.loadPage(0)
        page1_textblock = page1.getTextBlocks()
        page1_textblock = [x[4] for x in page1_textblock]

        section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
        section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]

        if len(section_start_index)==0:
            return pd.DataFrame({'CRD Number':[CRD_number],
                                 'Section Titles':['No Section'],
                                 'Section Pages':['No Section']})
        section_titles = page1_textblock[(section_start_index[0]+1):section_end_index[0]]

        section_pages = page1_textblock[(section_end_index[0]+1):(section_end_index[0]+len(section_titles)+1)]

        return pd.DataFrame({'CRD Number':[CRD_number]*len(section_titles),
                             'Section Titles':section_titles,
                             'Section Pages':section_pages})
    except:
        print(file)
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Section Titles':['No Section'],
                             'Section Pages':['No Section']})

In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_section_info_Pool,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

len(result)

In [ ]:
result_df = pd.concat(result)

In [ ]:
result_df.shape

In [ ]:
result_df

In [ ]:
result_df.to_csv('broker_section_title.csv',index = False)

In [11]:
def get_branch_office(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
    
    page_summary = fitz_doc.loadPage(2)
    page_summary_textblock = page_summary.getTextBlocks()
    page_summary_textblock = [x[4] for x in page_summary_textblock]
    
    if (any(['This broker is not currently registered' in x for x in page_summary_textblock]) or
        any(['This broker is not currently registered with' in x and 'FINRA firm' in x for x in page_summary_textblock])):
        Firm_CRD = 'NOT FOUND'
        Start_Date = 'NOT FOUND'
        Company_Info = 'NOT FOUND'
    elif any(['Currently employed by and registered with' in x for x in page_summary_textblock]):
        cutting_index = [i for i,x in enumerate(page_summary_textblock) if 'Currently employed by and registered with' in x][0]
        Firm_CRD = re.findall(r'(\d+)',[x for x in page_summary_textblock[cutting_index+1].split('\n') if 'CRD' in x][0])[0]
        Start_Date = re.findall(r'(\d+/\d+/\d+)',page_summary_textblock[cutting_index+1].split('CRD#')[-1])
        if len(Start_Date)==0:
            Start_Date = 'NOT FOUND'
        else:
            Start_Date = Start_Date[0]
        Company_Info = page_summary_textblock[cutting_index+1].split('CRD#')[0].replace('\n',' ').strip()
    return pd.DataFrame({'CRD Number':[CRD_number],
                         'Company Info':[Company_Info],
                         'Company CRD':[Firm_CRD],
                         'Date Since':[Start_Date]})
    print(page_summary_textblock)
    

In [ ]:
section_info_df = pd.DataFrame()

for file in broker_pdf_list:
    print(file)
    doc = fitz.open(file)
    section_info_df = pd.concat([section_info_df,get_section_info(doc)])

In [ ]:
section_info_df.head(50)

In [12]:
def separate_consecutive_numbers(num_list):
    group_list = []
    for k, g in groupby([(x, i-x) for i,x in enumerate(num_list)], lambda x:x[1]):
        group_list.append(list(g))
    group_list = [[(c[0],i) for c in x] for i,x in enumerate(group_list)]
    return group_list

In [13]:
def extract_state(x):
    try:
        state_value = ['ALABAMA','ALASKA','ARIZONA','ARKANSAS','CALIFORNIA','COLORADO','CONNECTICUT','DELAWARE',
                   'FLORIDA','GEORGIA','HAWAII','IDAHO','ILLINOIS','INDIANA','IOWA','KANSAS','KENTUCKY','LOUISIANA',
                   'MAINE','MARYLAND','MASSACHUSETTS','MICHIGAN','MINNESOTA','MISSISSIPPI','MISSOURI','MONTANA',
                   'NEBRASKA','NEVADA','NEW HAMPSHIRE','NEW JERSEY','NEW MEXICO','NEW YORK','NORTH CAROLINA',
                   'NORTH DAKOTA','OHIO','OKLAHOMA','OREGON','PENNSYLVANIA','RHODE ISLAND','SOUTH CAROLINA','SOUTH DAKOTA',
                   'TENNESSEE','TEXAS','UTAH','VERMONT','VIRGINIA','WASHINGTON','WEST VIRGINIA','WISCONSIN','WYOMING',
                   'GUAM','PUERTO RICO','VIRGIN ISLANDS','DISTRICT OF COLUMBIA']
        state_condidates = [c for c in state_value if c.lower() in x.lower()]
        state_condidates_length = [len(c) for c in state_condidates]
        return ' '.join([y.capitalize() for y in [c for c in state_condidates if len(c)==max(state_condidates_length)][0].split()])
    except:
        return 'NOT FOUND'

In [192]:
def text2int(textnum, numwords={}):
    if not numwords:
      units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
      ]

      tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

      scales = ["hundred", "thousand", "million", "billion", "trillion"]

      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)

    current = result = 0
    for word in textnum.split():
        if word not in numwords:
          raise Exception("Illegal word: " + word)

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0

    return result + current


In [195]:
text2int('one inc')

Exception: Illegal word: inc

In [199]:
w2n.word_to_num('ONE Plaza Five')

6

In [72]:
def get_broker_qual_employ_from_pagetextblock(page_textblock):
    #page_textblock = broker_page_textblock
    page_textblock = [x[4] for x in page_textblock]
    exam_section_start_index = [i for i,x in enumerate(page_textblock) if 'This individual has passed' in x or 'Industry Exams this Broker has Passed' in x][0]
    page_textblock = page_textblock[:exam_section_start_index]
    
    employment_check = [x for x in page_textblock if 'Employment' in x and 'of' in x]
    
    if len(employment_check)==0:
        broker_qual_employ = pd.DataFrame({'Table':['NOT FOUND'],
                                           'SRO or US Territory':['NOT FOUND'],
                                           'Category':['NOT FOUND'],
                                           'Status':['NOT FOUND'],
                                           'Date':['NOT FOUND']})
        return broker_qual_employ
    
    employment_num = max([int(c) for c in re.findall(r'\d+',[x for x in page_textblock if 'Employment' in x and 'of' in x][0])])
    #print(employment_num)
    employment_section_start_index = [i for i,x in enumerate(page_textblock) if 'Employment' in x and 'of' in x]
    #print(len(employment_num))
    if len(employment_section_start_index)==1:
        employment_section_end_index = [len(page_textblock)]
    else:
        employment_section_end_index = employment_section_start_index[1:] + [len(page_textblock)]
    
    broker_qual_employ = pd.DataFrame()
    
    for i in range(len(employment_section_start_index)):
        page_textblock_temp = page_textblock[employment_section_start_index[i]:employment_section_end_index[i]]
    
        SRO_table_start_index = [i for i,x in enumerate(page_textblock_temp) if all([c in x for c in ['SRO','Category','Date']])]
        US_Territory_table_start_index = [i for i,x in enumerate(page_textblock_temp) if 'U.S. State/\nTerritory' in x]

        if len(SRO_table_start_index)!=0:
            
            if len(SRO_table_start_index)==1:
                if len(US_Territory_table_start_index)==0:
                    SRO_table_end_index = [len(page_textblock_temp)]
                else:
                    SRO_table_end_index = [US_Territory_table_start_index[0]]
            else:
                if len(US_Territory_table_start_index)==0:
                    SRO_table_end_index = SRO_table_start_index[1:]+[len(page_textblock_temp)]
                else:
                    SRO_table_end_index = SRO_table_start_index[1:] + [US_Territory_table_start_index[0]]
            SRO_grouped = []
            for i in range(len(SRO_table_start_index)):
                #print(SRO_table_start_index[i],SRO_table_end_index[i])
                SRO_grouped.append(page_textblock_temp[SRO_table_start_index[i]:SRO_table_end_index[i]])
            #print(SRO_grouped)
            for SRO_grouped_each in SRO_grouped:
                SRO_block_dated = [(i,x) for i,x in enumerate(SRO_grouped_each) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
                for i,x in SRO_block_dated:
                    #print(i,x)
                    if len(re.sub(r'(\d+/\d+/\d+)','',SRO_grouped_each[i-1]))!=len(SRO_grouped_each[i-1]):
                        #print(x)
                        Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                        x = x.split(Date)[0]
                        SRO_or_US_Territory = x.split('\n')[0]
                        x = x.split(SRO_or_US_Territory)[-1]
                        #print(x)
                        Category = ' '.join([c for c in x.split() if c.capitalize()==c])
                        #print(Category)
                        Status = ' '.join([c for c in x.split(Category)[-1].split() if c.upper()==c])
                        broker_qual_employ_temp = pd.DataFrame({'Table':['SRO'],
                                                           'SRO or US Territory':[SRO_or_US_Territory],
                                                           'Category':[Category],
                                                           'Status':[Status],
                                                           'Date':[Date]})
                        broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
                    else:
                        if '\n' in x:
                            #print('check')
                            Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                            x = x.split(Date)[0]
                            if all([c in SRO_grouped_each[i-1] for c in ['SRO','Category','Date']]):
                                SRO_or_US_Territory = x.split('\n')[0]
                            else:
                                SRO_or_US_Territory = SRO_grouped_each[i-1].replace('\n',' ')
                            #x = x.split(SRO_or_US_Territory)[-1]
                            #print(x)
                            Category = ' '.join([c for c in x.split() if c.capitalize()==c])
                            Status = ' '.join([c for c in x.split(Category)[-1].split() if c.upper()==c])
                            broker_qual_employ_temp = pd.DataFrame({'Table':['SRO'],
                                                               'SRO or US Territory':[SRO_or_US_Territory],
                                                               'Category':[Category],
                                                               'Status':[Status],
                                                               'Date':[Date]})
                            broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
        if len(US_Territory_table_start_index)!=0:
            
            if len(US_Territory_table_start_index)==1:
                US_Territory_table_end_index = [len(page_textblock_temp)]
            else:
                US_Territory_table_end_index = US_Territory_table_start_index[1:]+[len(page_textblock_temp)]            
            
            US_Territory_grouped = []

            for i in range(len(US_Territory_table_start_index)):
                US_Territory_grouped.append(page_textblock_temp[US_Territory_table_start_index[i]:US_Territory_table_end_index[i]])

            for US_Territory_grouped_each in US_Territory_grouped:
                US_Territory_block_dated = [(i,x) for i,x in enumerate(US_Territory_grouped_each) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]

                for i,x in US_Territory_block_dated:
                    #print(i,x)
                    if extract_state(x)!='NOT FOUND':
                        #print(x)
                        Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                        x = x.split(Date)[0]
                        Status = x.split('\n')[-1].strip()
                        SRO_or_US_Territory = extract_state(x)
                        Category = x.split('\n')[0].split(SRO_or_US_Territory)[-1].strip()
                        broker_qual_employ_temp = pd.DataFrame({'Table':['US Territory'],
                                                                'SRO or US Territory':[SRO_or_US_Territory],
                                                                'Category':[Category],
                                                                'Status':[Status],
                                                                'Date':[Date]})
                        broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
                    else:
                        #print(x)
                        Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                        x = x.split(Date)[0]
                        Status = x.split('\n')[-1].strip()
                        SRO_or_US_Territory = extract_state(US_Territory_grouped_each[i-1].replace('\n',' '))

                        if SRO_or_US_Territory == 'NOT FOUND':
                            continue
                        else:
                            Category = x.split('\n')[0].split(SRO_or_US_Territory)[-1].strip()
                            broker_qual_employ_temp = pd.DataFrame({'Table':['US Territory'],
                                                                    'SRO or US Territory':[SRO_or_US_Territory],
                                                                    'Category':[Category],
                                                                    'Status':[Status],
                                                                    'Date':[Date]})
                            broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
    if broker_qual_employ.shape[0]==0:
        broker_qual_employ = pd.DataFrame({'Table':['NOT FOUND'],
                                           'SRO or US Territory':['NOT FOUND'],
                                           'Category':['NOT FOUND'],
                                           'Status':['NOT FOUND'],
                                           'Date':['NOT FOUND']})
    return broker_qual_employ

In [75]:
def get_broker_qual_employ_from_pagetextblock_Pool(file):
    try:
        #print(file)
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file)[0]
        #print(CRD_number)
        
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
            pages = [int(x) for x in broker_qualification_page.split('-')]

            broker_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            broker_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
        #print('Check')
        broker_qual_employ_temp =  get_broker_qual_employ_from_pagetextblock(broker_page_textblock)
        broker_qual_employ_temp['CRD Number'] = [CRD_number]*broker_qual_employ_temp.shape[0]
        #print(broker_qual_employ_temp)
        return broker_qual_employ_temp
    except:
        print(file + ' Failed')

In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_broker_qual_employ_from_pagetextblock_Pool,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

len(result)

mupdf: cannot recognize version marker
mupdf: no objects found


/home/ubuntu/Broker/individual_355047.pdf Failed
/home/ubuntu/Broker/individual_1390910.pdf Failed


In [ ]:
result_df = pd.concat(result)

In [ ]:
result_df.shape

In [ ]:
result_df

In [ ]:
result_df.to_csv('broker_SRO_US_Territory.csv',index = False)

In [13]:
def exam_class_identify_old(x):
    if all([c in x.lower() for c in ['principal','supervisory','exam']]):
        return 'Principal Supervisory Exam'
    elif all([c in x.lower() for c in ['general','industry','product','exam']]):
        return "General Industry Product Exam"
    else:
        return 'State Securities Law Exam'

In [14]:
def exam_class_identify_new(each_exam_group):
    x = ' '.join(each_exam_group)
    if all([c in x.lower() for c in ['principal','supervisory','exam']]):
        return 'Principal Supervisory Exam'
    elif all([c in x.lower() for c in ['general','industry','product','exam']]):
        return "General Industry Product Exam"
    else:
        return 'State Securities Law Exam'

In [15]:
def exam_name_identify_new(each_exam_group,i):
    if 'Examination' in each_exam_group[i]:
        return each_exam_group[i].split('Examination')[0] + ' Examination'
    else:
        x = ' '.join([each_exam_group[i-1], each_exam_group[i]])
        return x.split('Examination')[0] + ' Examination'

In [16]:
def get_broker_qual_exam_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    exam_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['this individual has passed']])]
    
    broker_qual_exam = pd.DataFrame()
    
    if len(exam_section_index)!=0:
        exam_count = re.findall(r'[0-9]',page_textblock[exam_section_index[0]])
        
        principal_supervisory_exam = exam_count[0]
        general_industry_product_exam = exam_count[1]
        state_securities_law_exam = exam_count[2]
        
        page_textblock = page_textblock[exam_section_index[0]:]

        exam_index = [i for i,x in enumerate(page_textblock) if all([c in x for c in ['Exam','Category','Date']])]

        exam_group = []
        
        for i,j in zip(exam_index,exam_index[1:]+[len(page_textblock)]):
            exam_group.append(page_textblock[i:j])
        #print(exam_group)
        
        
        
        for each_exam_group in exam_group:
            index_dated = [(i,x) for i,x in enumerate(each_exam_group) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
            #ach_exam_group_dated = each_exam_group[index_dated]
            if len(index_dated)==0:
                
                continue
            elif any([True if all([c in x for c in ['Class','Exam','Category','Date']]) else False for x in each_exam_group]):
                Date = [re.findall(r'(\d+/\d+/\d+)',each_exam_group[i])[0] for i,text in index_dated]
                Exam_Class = [exam_class_identify_old(each_exam_group[i]) for i,text in index_dated] 
                Exam_Name = [each_exam_group[i].split('Exams')[-1].split('Examination')[0]+' Examination' for i,text in index_dated]    
                Category = [re.sub(r'(\d+/\d+/\d+)','',each_exam_group[i].split('Examination')[-1].strip()).strip() for i,text in index_dated]
            else:
                Date = [re.findall(r'(\d+/\d+/\d+)',each_exam_group[i])[0] for i,text in index_dated]
                Exam_Class = [exam_class_identify_new(each_exam_group) for i,text in index_dated] 
                Exam_Name = [exam_name_identify_new(each_exam_group,i) for i,text in index_dated]    
                Category = [re.sub(r'(\d+/\d+/\d+)','CUTTING_POINT',each_exam_group[i]).split('CUTTING_POINT')[-1].strip() for i,text in index_dated]
            
            broker_qual_exam_temp = pd.DataFrame({'Exam Class':Exam_Class,
                                                  'Exam Name':Exam_Name,
                                                  'Category':Category,
                                                  'Date':Date})
            
            broker_qual_exam = pd.concat([broker_qual_exam,broker_qual_exam_temp])
        
    else:
        broker_qual_exam = pd.DataFrame()
        
    if broker_qual_exam.shape[0]==0:
        Date = ['NOT FOUND']
        Exam_Class = ['NOT FOUND']
        Exam_Name = ['NOT FOUND']
        Category = ['NOT FOUND']
        broker_qual_exam = pd.DataFrame({'Exam Class':Exam_Class,
                                                  'Exam Name':Exam_Name,
                                                  'Category':Category,
                                                  'Date':Date})
    return broker_qual_exam

In [17]:
def get_regist_hist_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    regist_hist_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['Registration History']])]
    employ_hist_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['Employment History']])]
    
    if len(regist_hist_section_index)!=0:
        if len(employ_hist_section_index)==0:
            employ_hist_section_index = [len(page_textblock)]
    
        page_textblock = page_textblock[regist_hist_section_index[0]:employ_hist_section_index[0]]
        
        index_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+)','',x))!=len(x)]
        
        Start_Date = [re.findall(r'(\d+/\d+)',x)[0] for i,x in index_dated]
        End_Date = [re.findall(r'(\d+/\d+)',x)[1] for i,x in index_dated]
        
    else:
        registration_history = pd.DataFrame()
        
    return registration_history

In [ ]:
# 816782

In [ ]:
broker_name_df = pd.DataFrame()
for file in broker_pdf_list:
    #print(file)
    doc = fitz.open(file)
    broker_name_df = pd.concat([broker_name_df,get_broker_name_CRD(doc)])

In [ ]:
broker_name_df.tail(50)

In [ ]:
def get_broker_qual_exam_from_pagetextblock_Pool(file):
    try:
        #print(file)
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file)[0]
        
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
            pages = [int(x) for x in broker_qualification_page.split('-')]

            broker_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            broker_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
        #print('Check')
        broker_qual_exam_temp =  get_broker_qual_exam_from_pagetextblock(broker_page_textblock)
        broker_qual_exam_temp['CRD Number'] = [CRD_number]*broker_qual_exam_temp.shape[0]
        #print(broker_qual_employ_temp)
        return broker_qual_exam_temp
    except:
        print(file)


In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_broker_qual_exam_from_pagetextblock_Pool,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

len(result)

In [ ]:
result_df = pd.concat(result)

In [ ]:
result_df.shape

In [ ]:
result_df

In [ ]:
result_df.to_csv('broker_exam.csv',index = False)

In [200]:
get(w2n.word_to_num('ONE'),'NOT NUMBER')

NameError: name 'get' is not defined

In [201]:
w2n.word_to_num('haha')

ValueError: No valid number words found! Please enter a valid number word (eg. two million twenty three thousand and forty nine)

In [ ]:
file = 

In [252]:
def check_first_word_number(x):
    try:
        w2n.word_to_num(x.split()[0])
        return 'NUMBER'
    except:
        return 'NOT NUMBER'

In [362]:
def add_next_line(x):
    x = x.split('\n')[0]
    if (x[0].isdigit()==False and 
        check_first_word_number(x)=='NOT NUMBER' and 
        all([c not in x.lower() for c in ['street']]) and
        all([c not in x for c in ['P.','O.','BOX']]) and
        all([c not in x for c in ['P.','O.','Box']])):
        return True
    else:
        return False

In [375]:
def get_branch_office_info(file):
    #print(file)
    doc = fitz.open(file)
    section_info_df_temp =  get_section_info(doc)
    CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
    if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
        broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Report Summary']['Section Pages'].iloc[0]
        pages = [int(x) for x in broker_qualification_page.split('-')]

        summary_page_textblock = []
        for page in range(min(pages),max(pages)+1):
            summary_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
    else:
        summary_page_textblock = []
        for page in range(doc.pageCount):
            if any(['Report Summary for this Broker' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                summary_page_textblock.extend(doc.loadPage(page).getTextBlocks())
            #print('Check')

    head_cutoff = min([int(i) for i,x in enumerate(summary_page_textblock) if 'Report Summary for this Broker' in x])

    branch_office_textblock = [x[4] for x in summary_page_textblock[:head_cutoff]]

    branch_office_df = pd.DataFrame()
    
    #print(branch_office_textblock)
    if any(['Currently employed by and registered with' in x for x in branch_office_textblock]):
        
        branch_office_end_index = [i+1 for i,x in enumerate(branch_office_textblock) if 'Registered with this firm since:' in x]
        
        if len(branch_office_end_index)==1:
            Register_Status = 'CURRENTED REGISTERED WITH FINRA'
            tail_cutoff = max([i for i,x in enumerate(branch_office_textblock) if 'Currently employed by and registered with' in x])
            branch_office = branch_office_textblock[(tail_cutoff+1):][0]
            
            Branch_Office_Name = branch_office.split('\n')[0]
            branch_name_index = 1
            #print(branch_office)
            #print(Branch_Office_Name)
            while add_next_line(branch_office.replace(Branch_Office_Name,'').replace('\n','')):
                #print(branch_office.replace(Branch_Office_Name,'').replace('\n',''))
                #print(Branch_Office_Name)
                Branch_Office_Name = '\n'.join(branch_office.split('\n')[:(branch_name_index+1)])
                branch_name_index+=1
                
            Branch_Office_CRD = re.findall('\d+',[x for x in branch_office.split('\n') if 'CRD#' in x][0])[0]
            Branch_Office_Start_Date = re.findall('\d+/\d+/\d+',[x for x in branch_office.split('\n') if 'Registered with this firm since:' in x][0])[0]
            Branch_Office_ADDRESS = branch_office.replace(Branch_Office_Name,'').replace(Branch_Office_CRD,'').replace('CRD#','').replace(Branch_Office_Start_Date,'').replace('Registered with this firm since:','').replace('\n','').strip()
            Branch_Office_Name = Branch_Office_Name.replace('\n',' ')
            
            branch_office_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                 'Register Status':[Register_Status],
                                                 'Branch Office Name':[Branch_Office_Name],
                                                 'Branch Office ADDRESS':[Branch_Office_ADDRESS],
                                                 'Branch Office CRD':[Branch_Office_CRD],
                                                 'Branch Office Start Date':[Branch_Office_Start_Date]})
            branch_office_df = pd.concat([branch_office_df,branch_office_df_temp])
        else:
            branch_office_start_index = [max([i for i,x in enumerate(branch_office_textblock) if 'Currently employed by and registered with' in x])]+branch_office_end_index[:-1]
            
            for i in range(len(branch_office_start_index)):
                
                branch_office_textblock_temp = branch_office_textblock[branch_office_start_index[i]:(branch_office_end_index[i]+1)]
                #print(branch_office_textblock_temp)
                Register_Status = 'CURRENTED REGISTERED WITH FINRA'
                if i==0:
                    tail_cutoff = max([i for i,x in enumerate(branch_office_textblock_temp) if 'Currently employed by and registered with' in x])
                    branch_office = branch_office_textblock_temp[(tail_cutoff+1):][0]
                else:
                    branch_office = branch_office_textblock_temp[0]
                Branch_Office_Name = branch_office.split('\n')[0]
                branch_name_index = 1
                while add_next_line(branch_office.replace(Branch_Office_Name,'').replace('\n','')):
                    #print(branch_office.replace(Branch_Office_Name,'').replace('\n',''))
                    #print(Branch_Office_Name)
                    Branch_Office_Name = '\n'.join(branch_office.split('\n')[:(branch_name_index+1)])
                    branch_name_index+=1
                Branch_Office_CRD = re.findall('\d+',[x for x in branch_office.split('\n') if 'CRD#' in x][0])[0]
                Branch_Office_Start_Date = re.findall('\d+/\d+/\d+',[x for x in branch_office.split('\n') if 'Registered with this firm since:' in x][0])[0]
                Branch_Office_ADDRESS = branch_office.replace(Branch_Office_Name,'').replace(Branch_Office_CRD,'').replace('CRD#','').replace(Branch_Office_Start_Date,'').replace('Registered with this firm since:','').replace('\n','').strip()
                
                Branch_Office_Name = Branch_Office_Name.replace('\n',' ')
                
                branch_office_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                 'Register Status':[Register_Status],
                                                 'Branch Office Name':[Branch_Office_Name],
                                                 'Branch Office ADDRESS':[Branch_Office_ADDRESS],
                                                 'Branch Office CRD':[Branch_Office_CRD],
                                                 'Branch Office Start Date':[Branch_Office_Start_Date]})
                branch_office_df = pd.concat([branch_office_df,branch_office_df_temp])
    elif any(['This individual is no longer registered\nwith FINRA' in x for x in branch_office_textblock]):
        Register_Status = 'NO LONGER REGISTERED WITH FINRA'
        Branch_Office_Name = 'NOT AVAILABLE'
        Branch_Office_ADDRESS = 'NOT AVAILABLE'
        Branch_Office_CRD = 'NOT AVAILABLE'
        Branch_Office_Start_Date = 'NOT AVAILABLE'
        
        branch_office_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                 'Register Status':[Register_Status],
                                                 'Branch Office Name':[Branch_Office_Name],
                                                 'Branch Office ADDRESS':[Branch_Office_ADDRESS],
                                                 'Branch Office CRD':[Branch_Office_CRD],
                                                 'Branch Office Start Date':[Branch_Office_Start_Date]})
        branch_office_df = pd.concat([branch_office_df,branch_office_df_temp])
    elif any(['This broker is not currently registered' in x for x in branch_office_textblock]):
        Register_Status = 'NO CURRENTLY REGISTERED'
        Branch_Office_Name = 'NOT AVAILABLE'
        Branch_Office_ADDRESS = 'NOT AVAILABLE'
        Branch_Office_CRD = 'NOT AVAILABLE'
        Branch_Office_Start_Date = 'NOT AVAILABLE'
        
        branch_office_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                 'Register Status':[Register_Status],
                                                 'Branch Office Name':[Branch_Office_Name],
                                                 'Branch Office ADDRESS':[Branch_Office_ADDRESS],
                                                 'Branch Office CRD':[Branch_Office_CRD],
                                                 'Branch Office Start Date':[Branch_Office_Start_Date]})
        branch_office_df = pd.concat([branch_office_df,branch_office_df_temp])

    return branch_office_df

In [376]:
file = 'C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2132865.pdf'

In [377]:
branch_office_df = pd.DataFrame()
for file in broker_pdf_list:
    branch_office_df = pd.concat([branch_office_df,get_branch_office_info(file)])

In [380]:
branch_office_df['Branch Office Name'].unique()

array(['NOT AVAILABLE', 'STIFEL, NICOLAUS & COMPANY, INCORPORATED',
       'CENTURY SECURITIES ASSOCIATES, INC.',
       'CREDIT SUISSE SECURITIES (USA) LLC', 'SECURITIES AMERICA, INC.',
       'NYLIFE SECURITIES LLC',
       'MERRILL LYNCH, PIERCE, FENNER & SMITH INCORPORATED',
       'FSC SECURITIES CORPORATION',
       'MORGAN STANLEY WASHINGTON TRUST FINANCIAL CTR',
       'MML INVESTORS SERVICES, LLC', 'CENTAURUS FINANCIAL, INC.',
       'AMERIPRISE FINANCIAL SERVICES, LLC',
       'THRIVENT INVESTMENT MANAGEMENT INC.',
       'CROWN CAPITAL SECURITIES, L.P.',
       'ROYAL ALLIANCE ASSOCIATES, INC.',
       'ALLSTATE FINANCIAL SERVICES, LLC',
       'CAMBRIDGE INVESTMENT RESEARCH, INC.', 'CANTELLA & CO., INC.',
       'ADVISORY GROUP EQUITY SERVICES',
       'ADVISORY GROUP EQUITY SERVICES LTD.', 'LPL FINANCIAL LLC',
       'WELLS FARGO CLEARING SERVICES, LLC', 'MORGAN STANLEY',
       'UBS FINANCIAL SERVICES INC.', 'JANNEY MONTGOMERY SCOTT LLC',
       'MOORS & CABOT, INC.', 'D.

In [379]:
[print(x) for x in branch_office_df['Branch Office ADDRESS']]

NOT AVAILABLE
501 N. BROADWAYST. LOUIS, MO  63102
501 NORTH BROADWAYST. LOUIS, MO  63102
11 MADISON AVENUEEQUITIESNEW YORK, NY  10010
12325 PORT GRACE BLVD.LAVISTA, NE  68128
32770 Grand RiverSuite B200Farmington, MI  48336
51 MADISON AVENUENEW YORK, NY  10010
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
225 FRANKLIN STRETIREMENT & PERSONAL WEALTHSOLUTIONSBOSTON, MA  02110
1100-1800 AMERICAN BLVDRETIREMENT & PERSONAL WEALTHSOLUTIONSPENNINGTON, NJ  08534
NOT AVAILABLE
646 NORTH FRENCH RD.SUITE 9AMHERST, NY  14228
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
717 W. SPRAGUE AVE. STE 500SPOKANE, WA  99201
1000 CORPORATE DRIVESUITE 600FORT LAUDERDALE, FL  33334
12536 UTOPIA WAYSAN DIEGO, CA  92128
2300 E. KATELLA AVENUESUITE #200ANAHEIM, CA  92806
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
NOT AVAILABLE
190 LITTLETON ROADSUITE 11WESTFORD, MA  01886
NOT AVAILAB

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [365]:
get_branch_office_info(file)

,CRD Number,Register_Status,Branch_Office_Name,Branch_Office_ADDRESS,Branch_Office_CRD,Branch_Office_Start_Date
0,2132865,CURRENTED REGISTERED WITH FINRA,"NPB FINANCIAL GROUP, LLC",OCEAN & MISSION STREET (NW CORNER)P. O. BOX 17...,137743,05/01/2008


In [352]:
doc = fitz.open(file)
section_info_df_temp =  get_section_info(doc)
CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
    broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Report Summary']['Section Pages'].iloc[0]
    pages = [int(x) for x in broker_qualification_page.split('-')]

    summary_page_textblock = []
    for page in range(min(pages),max(pages)+1):
        summary_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
else:
    summary_page_textblock = []
    for page in range(doc.pageCount):
        if any(['Report Summary for this Broker' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
            summary_page_textblock.extend(doc.loadPage(page).getTextBlocks())
        #print('Check')

head_cutoff = min([int(i) for i,x in enumerate(summary_page_textblock) if 'Report Summary for this Broker' in x])

branch_office_textblock = [x[4] for x in summary_page_textblock[:head_cutoff]]

branch_office_df = pd.DataFrame()

In [353]:
branch_office_textblock

['FRANK M. PIEPER',
 'CRD# 2132865',
 'Currently employed by and registered with the\nfollowing Firm(s):',
 'NPB FINANCIAL GROUP, LLC\nOCEAN & MISSION STREET (NW CORNER)\nP. O. BOX 1745\nCARMEL, CA  93921-1745\nCRD# 137743\nRegistered with this firm since: 05/01/2008']

In [354]:
usaddress.parse('NPB FINANCIAL GROUP, LLC\nOCEAN & MISSION STREET (NW CORNER)\nP. O. BOX 1745\nCARMEL, CA  93921-1745')

[('NPB', 'Recipient'),
 ('FINANCIAL', 'Recipient'),
 ('GROUP,', 'Recipient'),
 ('LLC\n', 'Recipient'),
 ('OCEAN', 'Recipient'),
 ('&', 'AddressNumber'),
 ('MISSION', 'StreetName'),
 ('STREET', 'StreetNamePostType'),
 ('(NW', 'StreetNamePostDirectional'),
 ('CORNER)\n', 'StreetNamePostType'),
 ('P.', 'USPSBoxType'),
 ('O.', 'USPSBoxType'),
 ('BOX', 'USPSBoxType'),
 ('1745\n', 'USPSBoxID'),
 ('CARMEL,', 'PlaceName'),
 ('CA', 'StateName'),
 ('93921-1745', 'ZipCode')]

In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_branch_office_info,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

len(result)

In [ ]:
result_df = pd.concat(result)

In [ ]:
result_df.shape

In [ ]:
result_df.head()

In [ ]:
result_df.to_csv('broker_branch_office_info.csv',index = False)

In [237]:
branch_office_df = pd.DataFrame()

In [238]:
for file in broker_pdf_list:
    branch_office_df = pd.concat([branch_office_df,get_branch_office_info(file)])

In [268]:
file = 'C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_31169.pdf'

In [269]:
doc = fitz.open(file)
section_info_df_temp =  get_section_info(doc)
CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]

In [270]:
section_info_df_temp

,CRD Number,Section Titles,Section Pages
0,31169,Report Summary,1
1,31169,Broker Qualifications,2 - 4
2,31169,Registration and Employment History,5 - 6


In [294]:
def get_registration_history(file):
    doc = fitz.open(file)
    section_info_df_temp =  get_section_info(doc)
    CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
    if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
        broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Registration and Employment History']['Section Pages'].iloc[0]
        pages = [int(x) for x in broker_qualification_page.split('-')]

        regist_hist_page_textblock = []
        for page in range(min(pages),max(pages)+1):
            regist_hist_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
    else:
        regist_hist_page_textblock = []
        for page in range(doc.pageCount):
            if any(['Registration History' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                regist_hist_page_textblock.extend(doc.loadPage(page).getTextBlocks())
            #print('Check')

    regist_hist_page_textblock = [x[4] for x in regist_hist_page_textblock]

    regist_hist_start_index = [int(i) for i,x in enumerate(regist_hist_page_textblock) if 'Registration History' in x]
    regist_hist_page_textblock = regist_hist_page_textblock[max(regist_hist_start_index):]

    employ_hist_start_index = [int(i) for i,x in enumerate(regist_hist_page_textblock) if 'Employment History' in x]

    if len(employ_hist_start_index)!=0:
        regist_hist_page_textblock = regist_hist_page_textblock[:min(employ_hist_start_index)]

    regist_hist_page_textblock_dated = [(i,x.replace('\n',' ')) for i,x in enumerate(regist_hist_page_textblock) if len(re.findall(r'\d+/\d+',x))]

    broker_regist_hist = pd.DataFrame()

    if len(regist_hist_page_textblock_dated)==0:
        Start_Date = 'NOT AVAILABLE'
        End_Date = 'NOT AVAILABLE'
        Company_Name = 'NOT AVAILABLE'
        Company_CRD_Number = 'NOT AVAILABLE'
        Branch_Location = 'NOT AVAILABLE'
        broker_regist_hist_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                'Start Date':[Start_Date],
                                                'End Date':[End_Date],
                                                'Company Name':[Company_Name],
                                                'Company CRD Number':[Company_CRD_Number],
                                                'Branch Location':[Branch_Location]})
        broker_regist_hist = pd.concat([broker_regist_hist,broker_regist_hist_temp])
    else:
        for i,x in regist_hist_page_textblock_dated:
            Start_Date = re.findall(r'\d+/\d+',x)[0]
            End_Date = re.findall(r'\d+/\d+',x)[-1]
            x_tail = x.split(End_Date)[-1]
            Company_CRD_Number = re.findall('\d+',x_tail)
            if len(Company_CRD_Number)==0:
                x_tail = x_tail +' '+regist_hist_page_textblock[i+1]
            Company_CRD_Number = re.findall('\d+',x_tail)[0]
            Branch_Location = x_tail.split(Company_CRD_Number)[-1].strip()
            Company_Name = x_tail.split(Company_CRD_Number)[0].strip()
            broker_regist_hist_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                    'Start Date':[Start_Date],
                                                    'End Date':[End_Date],
                                                    'Company Name':[Company_Name],
                                                    'Company CRD Number':[Company_CRD_Number],
                                                    'Branch Location':[Branch_Location]})
            broker_regist_hist = pd.concat([broker_regist_hist,broker_regist_hist_temp])
    return broker_regist_hist

In [303]:
file

'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816799.pdf'

In [300]:
doc = fitz.open(file)
section_info_df_temp =  get_section_info(doc)
CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]

In [9]:
city_state_table = pd.read_csv('C:/Users/i26184/Desktop/Broker/us_cities_states_counties.csv', sep = '|')

city_state_table['City'] = [str(x).lower() for x in city_state_table['City']]
city_state_table.reset_index(inplace = True, drop = True)

In [10]:
def match_city_state(x,city_state_table):
    city_state_table_citystatematch = city_state_table.loc[[i for i,city,state in zip(range(city_state_table.shape[0]),city_state_table['City'],city_state_table['State short']) if str(city).lower() in str(x).lower() and state == x.split(',')[-1].strip()],:]
        
    if city_state_table_citystatematch.shape[0]==0:
        
        city_state_table_citystatematch = city_state_table.loc[[i for i,city,state in zip(range(city_state_table.shape[0]),city_state_table['City alias'],city_state_table['State short']) if str(city).lower() in str(x).lower() and state == x.split(',')[-1].strip()],:]
    
        if city_state_table_citystatematch.shape[0]==0:
            return 'NO MATCHING','NO MATCHING'
    
        else:
            city_state_table_citystatematch['Match_Length'] = [len(str(city)+str(state)) for city, state in zip(city_state_table_citystatematch['City alias'],city_state_table_citystatematch['State short'])]
            max_match_length = max(city_state_table_citystatematch['Match_Length'])
            city_state_table_citystatematch = city_state_table_citystatematch.loc[city_state_table_citystatematch['Match_Length']==max_match_length]
            city =  city_state_table_citystatematch['City alias'].iloc[0]
            state = city_state_table_citystatematch['State short'].iloc[0]

            if str(city).upper() in x:
                return str(city).upper(), state
            elif str(city).title() in x:
                return str(city).title(), state
            elif str(city).capitalize() in x:
                return str(city).capitalize(), state
            else:
                return str(city), state
    else:
        city_state_table_citystatematch['Match_Length'] = [len(str(city)+str(state)) for city, state in zip(city_state_table_citystatematch['City'],city_state_table_citystatematch['State short'])]
        max_match_length = max(city_state_table_citystatematch['Match_Length'])
        city_state_table_citystatematch = city_state_table_citystatematch.loc[city_state_table_citystatematch['Match_Length']==max_match_length]
        city =  city_state_table_citystatematch['City'].iloc[0]
        state = city_state_table_citystatematch['State short'].iloc[0]
        
        if str(city).upper() in x:
            return str(city).upper(), state
        elif str(city).title() in x:
            return str(city).title(), state
        elif str(city).capitalize() in x:
            return str(city).capitalize(), state
        else:
            return str(city), state

In [115]:
def get_employ_hist(file):
    doc = fitz.open(file)
    section_info_df_temp =  get_section_info(doc)
    CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
    if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
        broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Registration and Employment History']['Section Pages'].iloc[0]
        pages = [int(x) for x in broker_qualification_page.split('-')]

        employ_hist_page_textblock = []
        for page in range(min(pages),max(pages)+1):
            employ_hist_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
    else:
        employ_hist_page_textblock = []
        for page in range(doc.pageCount):
            if any(['Employment History' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                employ_hist_page_textblock.extend(doc.loadPage(page).getTextBlocks())
            #print('Check')
    employ_hist_page_textblock = [x[4] for x in employ_hist_page_textblock]
    employ_hist_start_index = [int(i) for i,x in enumerate(employ_hist_page_textblock) if 'Employment Dates Employer Name Employer Location' in x]
    employ_hist_end_index = [int(i) for i,x in enumerate(employ_hist_page_textblock) if 'FINRA. All rights reserved.' in x]
    #print(employ_hist_end_index)
    if len(employ_hist_start_index)==0:
        Start_Date = 'NOT AVAILABLE'
        End_Date = 'NOT AVAILABLE'
        Company_Name = 'NOT AVAILABLE'
        Branch_Location = 'NOT AVAILABLE'
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Start Date':[Start_Date],
                             'End Date':[End_Date],
                             'Company Name':[Company_Name],
                             'Branch Location':[Branch_Location]})
    
    broker_employ_hist = pd.DataFrame()
    
    for employ_table_num in employ_hist_start_index:
        employ_hist_start_index_temp = employ_table_num
        employ_hist_end_index_temp = min([int(i) for i in employ_hist_end_index if i>employ_hist_start_index_temp])
    
        employ_hist_page_textblock_temp = employ_hist_page_textblock[employ_hist_start_index_temp:employ_hist_end_index_temp]
        
        if any(['Other Business Activities' in x for x in employ_hist_page_textblock_temp]):
            employ_hist_end_index_temp = min([int(i) for i,x in enumerate(employ_hist_page_textblock_temp) if 'Other Business Activit' in x])
        
        employ_hist_page_textblock_temp = employ_hist_page_textblock_temp[:employ_hist_end_index_temp]
        #print(employ_hist_page_textblock_temp)
        employ_hist_page_textblock_dated = [(i,x.replace('\n',' ')) for i,x in enumerate(employ_hist_page_textblock_temp) if len(re.findall(r'\d+/\d+',x))]
        

        if len(employ_hist_page_textblock_dated)==0:
            Start_Date = 'NOT AVAILABLE'
            End_Date = 'NOT AVAILABLE'
            Company_Name = 'NOT AVAILABLE'
            Branch_Location = 'NOT AVAILABLE'
            broker_employ_hist_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                    'Start Date':[Start_Date],
                                                    'End Date':[End_Date],
                                                    'Company Name':[Company_Name],
                                                    'Branch Location':[Branch_Location]})
            broker_employ_hist = pd.concat([broker_employ_hist,broker_employ_hist_temp])
        else:
            for i,x in employ_hist_page_textblock_dated:
                Date_Candidates = re.findall(r'\d+/\d+',x)
                Start_Date = Date_Candidates[0]
                if len(Date_Candidates)==1:
                    End_Date = 'Present'
                else:
                    End_Date = Date_Candidates[-1]
                x_tail = x.split(End_Date)[-1].strip()
                x_tail = ' '.join(x.split())
                if match_city_state(x_tail,city_state_table) == ('NO MATCHING','NO MATCHING') and i+1 not in [x[0] for x in employ_hist_page_textblock_dated]:
                    #print(x_tail)
                    x_tail = x_tail +' '+employ_hist_page_textblock_temp[i+1]
                    #print(x_tail)
                city, state = match_city_state(x_tail,city_state_table)
                Branch_Location = ', '.join([city, state])
                #print(Branch_Location)
                #Company_Name = x_tail.split(End_Date)[-1].replace(city,'').replace(state,'').split(',')[0]
                Company_Name = ','.join(re.sub(str(state)+'$','',x_tail.split(End_Date)[-1].replace(city,'').strip()).split(',')[:-1])
                broker_employ_hist_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                        'Start Date':[Start_Date],
                                                        'End Date':[End_Date],
                                                        'Company Name':[Company_Name],
                                                        'Branch Location':[Branch_Location]})
                broker_employ_hist = pd.concat([broker_employ_hist,broker_employ_hist_temp])
    return broker_employ_hist

In [112]:
broker_employ_hist = pd.DataFrame()

for file in broker_pdf_list:
    print(file)
    broker_employ_hist = pd.concat([broker_employ_hist,get_employ_hist(file)])

C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_1390910.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2132865.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2400179.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2498722.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2809371.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_31169.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_3233747.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_704717.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816404.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816436.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816439.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816452.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816461.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816467.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816472.pdf
C:/Users/i26184/Desktop/Broker/Brok

In [89]:
file = 'C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_31169.pdf'

In [90]:
get_employ_hist(file)

,CRD Number,Start Date,End Date,Company Name,Branch Location
0,31169,08/2018,Present,Arbor Point Advisors,"Farmington, MI"
0,31169,06/2014,Present,SECURITIES AMERICA INC HILLS,"FARMINGTON, MI"
0,31169,04/1985,Present,BRODE FINANCIAL GROUP,"SOUTHFIELD, MI"
0,31169,01/2011,06/2014,DALTON STRATEGIC VESTMENT SERVICES,"KNIGHTSTOWN, IN"
0,31169,05/1990,12/2010,HAAS FINANCIAL PRODUCTS,"SOUTHFIELD, MI"


In [113]:
broker_employ_hist

,CRD Number,Start Date,End Date,Company Name,Branch Location
0,1390910,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE
0,2132865,05/2008,Present,"NPB FINANCIAL GROUP, LLC","CARMEL, CA"
0,2400179,05/2011,Present,"CENTURY SECURITIES ASSOCIATES, INC","FLORHAM PARK, NJ"
0,2400179,08/2007,Present,"STIFEL, NICOLAUS & COMPANY, INCORPORATED","FLORHAM PARK, NJ"
0,2498722,07/2016,Present,Credit Suisse Securities (USA) LLC,"New York, NY"
...,...,...,...,...,...
0,816796,02/1999,Present,WEDBUSH MORGAN SECURITIES INC.,"PORTLAND, OR"
0,816797,01/1979,Present,UBS FINANCIAL SERVICES INC.,"PORTLAND, OR"
0,816797,12/1978,Present,PAINEWEBBER INCORPORATED,"PORTLAND, OR"
0,816799,10/2015,Present,"Lave Avenue Financial, LLC","Pasadena, CA"


In [114]:
broker_employ_hist[broker_employ_hist['CRD Number']=='31169']

,CRD Number,Start Date,End Date,Company Name,Branch Location
0,31169,08/2018,Present,Arbor Point Advisors,"Farmington, MI"
0,31169,06/2014,Present,SECURITIES AMERICA INC HILLS,"FARMINGTON, MI"
0,31169,04/1985,Present,BRODE FINANCIAL GROUP,"SOUTHFIELD, MI"
0,31169,01/2011,06/2014,"DALTON STRATEGIC INVESTMENT SERVICES, INC","KNIGHTSTOWN, IN"
0,31169,05/1990,12/2010,"HAAS FINANCIAL PRODUCTS, INC.","SOUTHFIELD, MI"


In [ ]:
def get_other_business_activities()

In [533]:
file = 'C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816797.pdf'

In [544]:
other_activity_df = pd.DataFrame()

for file in broker_pdf_list:
    print(file)
    other_activity_df = pd.concat([other_activity_df,get_other_business_activities(file)])

C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_1390910.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2400179.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2498722.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_31169.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_3233747.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_704717.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816404.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816436.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816439.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816452.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816461.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816467.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816472.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816473.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816494.pdf
C:/Users/i26184/Desktop/Broker/Broker

In [609]:
file = 'C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2809371.pdf'

In [610]:
get_other_business_activities(file)

C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2809371.pdf


,CRD Number,Activity Detail
0,2809371,NOT FOUND


In [611]:
print(file)
doc = fitz.open(file)
section_info_df_temp =  get_section_info(doc)
CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
    regist_employ_hist_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Registration and Employment History']['Section Pages'].iloc[0]
    pages = [int(x) for x in regist_employ_hist_page.split('-')]

    other_activity_page_textblock = []
    for page in range(min(pages),max(pages)+1):
        other_activity_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
else:
    other_activity_page_textblock = []
    for page in range(doc.pageCount):
        if any(['Other Business Activit' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
            other_activity_page_textblock.extend(doc.loadPage(page).getTextBlocks())

other_activity_page_textblock = [x[4] for x in other_activity_page_textblock]
#print(other_activity_page_textblock)
other_activity_start_index = [i for i,x in enumerate(other_activity_page_textblock) if 'Other Business Activit' in x]
#print(other_activity_start_index)
other_activity_df = pd.DataFrame()

if len(other_activity_start_index)==0:
    Activity_Detail = 'NOT FOUND'
    other_activity_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                           'Activity Detail':[Activity_Detail]})
    other_activity_df = pd.concat([other_activity_df,other_activity_df_temp])
else:
    if 'No information reported' in other_activity_page_textblock[other_activity_start_index[0]+2]:
        Activity_Detail = 'NOT FOUND'
        other_activity_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                               'Activity Detail':[Activity_Detail]})
        other_activity_df = pd.concat([other_activity_df,other_activity_df_temp])
    else:
        other_activity_end_index = [i for i,x in enumerate(other_activity_page_textblock) if 'FINRA. All rights reserved' in x]
        #print(other_activity_end_index)
        for i in range(len(other_activity_start_index)):
            #print(other_activity_start_index[i],other_activity_end_index[i])
            if i==0:
                other_activity_page_textblock_temp = other_activity_page_textblock[(other_activity_start_index[i]+2):other_activity_end_index[i]]
            else:
                other_activity_page_textblock_temp = other_activity_page_textblock[(other_activity_start_index[i]+1):other_activity_end_index[i]]
            #print(other_activity_page_textblock_temp)
            for j in range(len(other_activity_page_textblock_temp)):
                Activity_Detail = other_activity_page_textblock_temp[j]
                other_activity_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                   'Activity Detail':[Activity_Detail]})
                other_activity_df = pd.concat([other_activity_df,other_activity_df_temp])
        other_activity_df.drop_duplicates(subset = ['Activity Detail'], inplace = True)

C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2809371.pdf


IndexError: list index out of range

In [612]:
other_activity_start_index

[11, 13]

In [613]:
other_activity_end_index

[14]

In [629]:
def get_other_business_activities(file):
    try:
        print(file)
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            regist_employ_hist_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Registration and Employment History']['Section Pages'].iloc[0]
            pages = [int(x) for x in regist_employ_hist_page.split('-')]

            other_activity_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                other_activity_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            other_activity_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Other Business Activit' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    other_activity_page_textblock.extend(doc.loadPage(page).getTextBlocks())

        other_activity_page_textblock = [x[4] for x in other_activity_page_textblock]
        #print(other_activity_page_textblock)
        other_activity_start_index = [i for i,x in enumerate(other_activity_page_textblock) if 'Other Business Activit' in x]
        #print(other_activity_start_index)
        other_activity_df = pd.DataFrame()

        if len(other_activity_start_index)==0:
            Activity_Detail = 'NOT FOUND'
            other_activity_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                   'Activity Detail':[Activity_Detail]})
            other_activity_df = pd.concat([other_activity_df,other_activity_df_temp])
        else:
            if 'No information reported' in other_activity_page_textblock[other_activity_start_index[0]+2]:
                Activity_Detail = 'NOT FOUND'
                other_activity_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                       'Activity Detail':[Activity_Detail]})
                other_activity_df = pd.concat([other_activity_df,other_activity_df_temp])
            else:
                other_activity_end_index = [i for i,x in enumerate(other_activity_page_textblock) if 'FINRA. All rights reserved' in x]
                other_activity_start_index = other_activity_start_index[:len(other_activity_end_index)]
                #print(other_activity_end_index)
                for i in range(len(other_activity_start_index)):
                    #print(other_activity_start_index[i],other_activity_end_index[i])
                    if i==0:
                        other_activity_page_textblock_temp = other_activity_page_textblock[(other_activity_start_index[i]+2):other_activity_end_index[i]]
                    else:
                        other_activity_page_textblock_temp = other_activity_page_textblock[(other_activity_start_index[i]+1):other_activity_end_index[i]]
                    #print(other_activity_page_textblock_temp)
                    for j in range(len(other_activity_page_textblock_temp)):
                        if 'Other Business Activit' in other_activity_page_textblock_temp[j]:
                            Activity_Detail_list = other_activity_page_textblock_temp[j].split('Other Business Activities')
                            other_activity_df_temp = pd.DataFrame({'CRD Number':[CRD_number]*len(Activity_Detail_list),
                                                               'Activity Detail':Activity_Detail_list})
                            other_activity_df = pd.concat([other_activity_df,other_activity_df_temp])
                        else:
                            Activity_Detail = other_activity_page_textblock_temp[j]
                            other_activity_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                               'Activity Detail':[Activity_Detail]})
                            other_activity_df = pd.concat([other_activity_df,other_activity_df_temp])
                            other_activity_df.drop_duplicates(subset = ['Activity Detail'], inplace = True)
        return other_activity_df
    except:
        CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
        Activity_Detail = 'NOT FOUND'
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Activity Detail':[Activity_Detail]})

In [630]:
other_activity_df = pd.DataFrame()

for file in broker_pdf_list:
    #print(file)
    other_activity_df = pd.concat([other_activity_df,get_other_business_activities(file)])

C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_1390910.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2400179.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2498722.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_31169.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_3233747.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_704717.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816404.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816436.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816439.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816452.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816461.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816467.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816472.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816473.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816494.pdf
C:/Users/i26184/Desktop/Broker/Broker

In [631]:
other_activity_df

,CRD Number,Activity Detail
0,1390910,NOT FOUND
0,2400179,NOT FOUND
0,2498722,NOT FOUND
0,31169,"DBA - BRODE FINANCIAL GROUP, INC."
0,31169,INSURANCE SALES - AGENT - INVESTMENT RELATED -...
...,...,...
0,816791,NOT FOUND
0,816793,NOT FOUND
0,816796,NOT FOUND
0,816797,"NASD DISPUTE RESOLUTION, 300 SOUTH GRAND\nAVEN..."


In [515]:
def get_disclosure_event_count(file):
    try:
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Report Summary']['Section Pages'].iloc[0]
            pages = [int(x) for x in broker_qualification_page.split('-')]

            summary_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                summary_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            summary_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Report Summary for this Broker' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    summary_page_textblock.extend(doc.loadPage(page).getTextBlocks())
                #print('Check')
        summary_page_textblock = [x[4] for x in summary_page_textblock]
        #print(summary_page_textblock)
        disclosure_info = [x for x in summary_page_textblock if 'Are there events disclosed about this broker?' in x][0]
        disclosure_info = disclosure_info.split('Are there events disclosed about this broker?')[-1].strip()

        disclosure_info_df = pd.DataFrame()

        if disclosure_info == 'No':
            Disclosure_Check = 'No'
            Disclosure_Type = 'NOT AVAILABLE'
            Disclosure_Num = 'NOT AVAILABLE'
            disclosure_info_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                   'Disclosure Check':[Disclosure_Check],
                                                   'Disclosure Type':[Disclosure_Type],
                                                   'Disclosure Num':[Disclosure_Num]})
            disclosure_info_df = pd.concat([disclosure_info_df, disclosure_info_df_temp])
        else:
            disclosure_detail_start_index = [i for i,x in enumerate(summary_page_textblock) if 'The following types of disclosures' in x or 'The following type of disclosure' in x][0]

            #print(disclosure_detail_start_index)
            disclosure_detail = summary_page_textblock[(disclosure_detail_start_index+1)]

            Disclosure_Check = 'Yes'
            Disclosure_Type = disclosure_detail.split('\n')[0]
            Disclosure_Num = disclosure_detail.split('\n')[-1]
            disclosure_info_df_temp = pd.DataFrame({'CRD Number':[CRD_number],
                                                   'Disclosure Check':[Disclosure_Check],
                                                   'Disclosure Type':[Disclosure_Type],
                                                   'Disclosure Num':[Disclosure_Num]})
            disclosure_info_df = pd.concat([disclosure_info_df, disclosure_info_df_temp])
        return disclosure_info_df
    except:
        CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
        Disclosure_Check = 'No'
        Disclosure_Type = 'NOT AVAILABLE'
        Disclosure_Num = 'NOT AVAILABLE'
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Disclosure Check':[Disclosure_Check],
                             'Disclosure Type':[Disclosure_Type],
                             'Disclosure Num':[Disclosure_Num]})

In [516]:
disclosure_info_df

,CRD Number,Disclosure Check,Disclosure Type,Disclosure Num
0,1390910,Yes,Regulatory Event,Regulatory Event
0,2400179,No,NOT AVAILABLE,NOT AVAILABLE
0,2498722,No,NOT AVAILABLE,NOT AVAILABLE
0,31169,No,NOT AVAILABLE,NOT AVAILABLE
0,3233747,No,NOT AVAILABLE,NOT AVAILABLE
...,...,...,...,...
0,816791,Yes,Criminal,1
0,816793,Yes,Regulatory Event,1
0,816796,Yes,Customer Dispute,1
0,816797,No,NOT AVAILABLE,NOT AVAILABLE


In [517]:
disclosure_info_df = pd.DataFrame()
for file in broker_pdf_list:
    print(file)
    disclosure_info_df = pd.concat([disclosure_info_df,get_disclosure_event_count(file)])

C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_1390910.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2400179.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_2498722.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_31169.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_3233747.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_704717.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816404.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816436.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816439.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816452.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816461.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816467.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816472.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816473.pdf
C:/Users/i26184/Desktop/Broker/Broker_PDF\individual_816494.pdf
C:/Users/i26184/Desktop/Broker/Broker

In [518]:
disclosure_info_df

,CRD Number,Disclosure Check,Disclosure Type,Disclosure Num
0,1390910,Yes,Regulatory Event,Regulatory Event
0,2400179,No,NOT AVAILABLE,NOT AVAILABLE
0,2498722,No,NOT AVAILABLE,NOT AVAILABLE
0,31169,No,NOT AVAILABLE,NOT AVAILABLE
0,3233747,No,NOT AVAILABLE,NOT AVAILABLE
...,...,...,...,...
0,816791,Yes,Criminal,1
0,816793,Yes,Regulatory Event,1
0,816796,Yes,Customer Dispute,1
0,816797,No,NOT AVAILABLE,NOT AVAILABLE


In [519]:
disclosure_info_df[disclosure_info_df['Disclosure Check']=='Yes']

,CRD Number,Disclosure Check,Disclosure Type,Disclosure Num
0,1390910,Yes,Regulatory Event,Regulatory Event
0,704717,Yes,Regulatory Event,Regulatory Event
0,816404,Yes,Customer Dispute,1
0,816439,Yes,Regulatory Event,Regulatory Event
0,816452,Yes,Regulatory Event,2
0,816500,Yes,Criminal,1
0,816514,Yes,Regulatory Event,Regulatory Event
0,816517,Yes,Judgment/Lien,1
0,816523,Yes,Customer Dispute,1
0,816526,Yes,Regulatory Event,1
